# mesures_cat_seuil
***

Essayons de mettre en pratique la proposition de Thomas dans la PR 89<br>
https://github.com/dataforgoodfr/13_pollution_eau/pull/89


# Proposition
***
Je pense qu'il serait intéressant d'avoir le stables intémédiares suivantes<br>
1. une table pour les données géo : comme udi_cog<br>
2. une table un par test/resultat avec les regexp et les catégories<br>
3. une autre table avec les données par communes<br>
4. une autre table avec les données par depatement <br>
5. une autre table avec les données par région<br>

/!\ Pour le traitement des données par région/commune/departement, il ne faut pas compter les resultats ou faire des pourcentages car un même résultat peut être utilisés plusieurs fois.
Il faudra juste avoir des données plus globale sur la qualité de l'eau : conforme, avec/sans pestidices, etc...

# Questions ouvertes
***

**Dans edc_communes, un cdreseau peut être lié à plusieurs communes et inversement.<br>
Est-ce attendu ?**<br>

Si je relis la partie "DESCRIPTIF DES DONNEES MISES A DISPOSITION" du pdf :<br>
" Les UDI peuvent englober les réseaux de distribution d’eau dans tout ou partie d’une ou plusieurs communes, suivant la situation locale.<br>
[...] Le fichier UDI_COM détaille le lien entre UDI et communes et permet de pouvoir reconstituer un accès aux résultats par commune ; il est relié au fichier PLV par l’intermédiaire du champ ‘cdreseau’, correspondant au code SISE-Eaux de distribution."
<br> ==> J'ai l'impression que oui


**Si on fait les jointures comme recommandé dans la documentation pour avoir les informations de communes/géographie,
on duplique les nombres de lignes de résultats. 
Un même résultats/prélèvement peut être utilisé pour plusieurs communes.**<br>
--> Est-ce qu'on est ok avec ça ?

_Dans la table edc_resultats on a 64 164 498 lignes_

Voici différents tests
- sub_query = Check avant jointure par cdreseau en utilisant prelevement_unique : même nombre de lignes que edc_resultats
- sub_query2 = Check avant jointure par cdreseau en utilisant edc_prelevement : 33 315 337 lignes en plus
- query = sub_query + Jointe par cdreseau sur communes : 581 458 710 lignes en plus
- query2 = sub_query + Jointe par cdreseau sur communes : 1 784 283 587 lignes en plus


Pour l'analyse des données de prélèvement est-ce qu'on veut garder tous les cdreseaux ou le plus en amont/ le "first" suffit ?<br>
--> Je pense qu'on veut garder toutes les lignes cdreseau

**Je pense que la table intermédiare que l'on souhaite avec les info par communes est la query2 mais il faut peut-être traiter les informations avec prudence car on a dupliquer des résultats (du à la situation locale)**


# Exploration
***

In [1]:
import duckdb
from pipelines.tasks.config.common import DUCKDB_FILE

import pandas as pd

pd.set_option("display.max_columns", None)  # show all cols
pd.set_option("display.max_colwidth", None)  # show full width of showing cols
pd.set_option(
    "display.expand_frame_repr", False
)  # print cols side by side as it's supposed to be

con = duckdb.connect(database=DUCKDB_FILE, read_only=True)

In [2]:
# Tables dispo dans la bdd
con.sql("SHOW TABLES").show()

┌─────────────────────────┐
│          name           │
│         varchar         │
├─────────────────────────┤
│ cog_communes            │
│ edc_communes            │
│ edc_prelevements        │
│ edc_resultats           │
│ laposte_communes        │
│ mapping_categories      │
│ stg_communes__cog       │
│ stg_communes__laposte   │
│ stg_edc__communes       │
│ stg_edc__prevelevements │
│ stg_edc__resultats      │
├─────────────────────────┤
│         11 rows         │
└─────────────────────────┘



In [3]:
nb_result_ref = con.sql(" SELECT COUNT(*) FROM edc_resultats").df()
nb_result_ref

,count_star()
0,64164498


In [4]:
query_prelevement_unique = """
WITH
prelevements_cdfirstreseauamont AS (
    SELECT DISTINCT
        referenceprel,
        dateprel,
        heureprel,
        conclusionprel,
        plvconformitebacterio,
        plvconformitechimique,
        plvconformitereferencebact,
        plvconformitereferencechim,
        (CASE
            WHEN cdreseauamont IS NULL THEN cdreseau
            WHEN cdreseauamont IS NOT NULL THEN cdreseauamont
        END) AS cdfirstreseauamont,
        TRY_STRPTIME(
            dateprel || ' ' || REPLACE(heureprel, 'h', ':'), '%Y-%m-%d %H:%M'
        ) AS datetimeprel
    FROM
        edc_prelevements
),

ranked AS (
    SELECT
        *,
        ROW_NUMBER() OVER (
            PARTITION BY referenceprel
            ORDER BY
                dateprel,
                heureprel
        ) AS row_num
    FROM
        prelevements_cdfirstreseauamont
)

SELECT * EXCLUDE (row_num)
FROM
    ranked
WHERE
    row_num = 1
"""

prelevement_unique = con.sql(query_prelevement_unique).df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## 1. Check avant jointure par cdreseau en utilisant prelevement_unique

In [5]:
sub_query = """
WITH
prelevement AS (
	SELECT
	  referenceprel,
	  cdfirstreseauamont AS cdreseau,
      dateprel,
	FROM   
      prelevement_unique
),

resultats AS (
    SELECT
      referenceprel,
      cdparametresiseeaux,
      valtraduite,
      limitequal,
      CAST(regexp_extract(REPLACE(limitequal, ',', '.'), '-?\d+(\.\d+)?') AS FLOAT) AS limitequal_float,
      regexp_extract(limitequal, '[a-zA-Zµg]+/?[a-zA-Z/L]+$') AS unite,
    FROM  
        edc_resultats 
    ),

mapping_categories_simple AS (
    SELECT
      cdparametresiseeaux  ,
      STRING_AGG(DISTINCT categorie) AS categorie,
    FROM 
      mapping_categories
    GROUP BY 
        cdparametresiseeaux
     HAVING 
        COUNT(DISTINCT categorie) = 1
),

mesures_cat AS (
    SELECT
        resultats.*,
        prelevement.dateprel,
        mapping_categories_simple.categorie
    FROM 
        resultats
    LEFT JOIN
    	prelevement
    ON
       resultats.referenceprel = prelevement.referenceprel
    LEFT JOIN 
        mapping_categories_simple
    ON
        resultats.cdparametresiseeaux = mapping_categories_simple.cdparametresiseeaux
    )
    
SELECT 
 COUNT(*)
FROM
 mesures_cat
"""

nb_result_sub_query = con.sql(sub_query).df()

if nb_result_sub_query.iloc[0].values == nb_result_ref.iloc[0].values:
    print("ok :)")
else:
    print("KO")
    print("diff: ", nb_result_sub_query.iloc[0].values - nb_result_ref.iloc[0].values)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Vinca\AppData\Local\Temp\ipykernel_14360\4281527644.py:1: SyntaxWarning: invalid escape sequence '\d'
  sub_query = """


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

ok :)


## 2. Check avant jointure par cdreseau en utilisant edc_prelevement

In [6]:
sub_query2 = """
WITH
prelevement AS (
	SELECT
	  referenceprel,
	  cdreseau,
      dateprel,
	FROM   
      edc_prelevements
),

resultats AS (
    SELECT
      referenceprel,
      cdparametresiseeaux,
      valtraduite,
      limitequal,
      CAST(regexp_extract(REPLACE(limitequal, ',', '.'), '-?\d+(\.\d+)?') AS FLOAT) AS limitequal_float,
      regexp_extract(limitequal, '[a-zA-Zµg]+/?[a-zA-Z/L]+$') AS unite,
    FROM  
        edc_resultats 
    ),

mapping_categories_simple AS (
    SELECT
      cdparametresiseeaux  ,
      STRING_AGG(DISTINCT categorie) AS categorie,
    FROM 
      mapping_categories
    GROUP BY 
        cdparametresiseeaux
     HAVING 
        COUNT(DISTINCT categorie) = 1
),

mesures_cat AS (
    SELECT
        resultats.*,
        prelevement.dateprel,
        mapping_categories_simple.categorie
    FROM 
        resultats
    LEFT JOIN
    	prelevement
    ON
       resultats.referenceprel = prelevement.referenceprel
    LEFT JOIN 
        mapping_categories_simple
    ON
        resultats.cdparametresiseeaux = mapping_categories_simple.cdparametresiseeaux
    )
    
SELECT 
 COUNT(*)
FROM
 mesures_cat
"""

nb_result_sub_query2 = con.sql(sub_query2).df()

if nb_result_sub_query2.iloc[0].values == nb_result_ref.iloc[0].values:
    print("ok :)")
else:
    print("KO")
    print("diff: ", nb_result_sub_query2.iloc[0].values - nb_result_ref.iloc[0].values)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Vinca\AppData\Local\Temp\ipykernel_14360\2566975602.py:1: SyntaxWarning: invalid escape sequence '\d'
  sub_query2 = """


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

KO
diff:  [33315337]


## 3. Avec Join sur edc_communes et cog_communes à partir du resultat 1

In [7]:
query = """
WITH
udi AS (
    SELECT
      inseecommune AS commune_code_insee,
      cdreseau,
    FROM
      edc_communes
),

cog AS (
    SELECT
      DEP AS code_departement,
      REG AS code_region,
      COM AS commune_code_insee,
    FROM 
      cog_communes
),

udi_cog AS (
    SELECT
      udi.commune_code_insee ,
      udi.cdreseau,
      cog.code_departement,
      cog.code_region
    FROM
      udi
    LEFT JOIN 
      cog
    ON 
      udi.commune_code_insee = cog.commune_code_insee
),

prelevement AS (
	SELECT
	  referenceprel,
	  cdfirstreseauamont AS cdreseau,
      dateprel,
	FROM   
      prelevement_unique
),

resultats AS (
    SELECT
      referenceprel,
      cdparametresiseeaux,
      valtraduite,
      limitequal,
      CAST(regexp_extract(REPLACE(limitequal, ',', '.'), '-?\d+(\.\d+)?') AS FLOAT) AS limitequal_float,
      regexp_extract(limitequal, '[a-zA-Zµg]+/?[a-zA-Z/L]+$') AS unite,
    FROM  
        edc_resultats 
    ),

mapping_categories_simple AS (
    SELECT
      cdparametresiseeaux  ,
      STRING_AGG(DISTINCT categorie) AS categorie,
    FROM 
      mapping_categories
    GROUP BY 
        cdparametresiseeaux
     HAVING 
        COUNT(DISTINCT categorie) = 1
),

mesures_cat AS (
    SELECT
        resultats.*,
        prelevement.dateprel,
        mapping_categories_simple.categorie
    FROM 
        resultats
    LEFT JOIN
    	prelevement
    ON
       resultats.referenceprel = prelevement.referenceprel
    LEFT JOIN
        udi_cog
    ON
    	udi_cog.cdreseau = prelevement.cdreseau
    LEFT JOIN 
        mapping_categories_simple
    ON
        resultats.cdparametresiseeaux = mapping_categories_simple.cdparametresiseeaux
    )
    
SELECT 
 COUNT(*)
FROM
 mesures_cat
"""

nb_result_1 = con.sql(query).df()

if nb_result_1.iloc[0].values == nb_result_ref.iloc[0].values:
    print("ok :)")
else:
    print("KO")
    print("diff: ", nb_result_1.iloc[0].values - nb_result_ref.iloc[0].values)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Vinca\AppData\Local\Temp\ipykernel_14360\972818049.py:1: SyntaxWarning: invalid escape sequence '\d'
  query = """


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

KO
diff:  [581458710]


## 4. Avec Join sur edc_communes et cog_communes à partir du resultat 2

In [8]:
query2 = """
WITH
udi AS (
    SELECT
      inseecommune AS commune_code_insee,
      cdreseau,
    FROM
      edc_communes
),

cog AS (
    SELECT
      DEP AS code_departement,
      REG AS code_region,
      COM AS commune_code_insee,
    FROM 
      cog_communes
),

udi_cog AS (
    SELECT
      udi.commune_code_insee ,
      udi.cdreseau,
      cog.code_departement,
      cog.code_region
    FROM
      udi
    LEFT JOIN 
      cog
    ON 
      udi.commune_code_insee = cog.commune_code_insee
),

prelevement AS (
	SELECT
	  referenceprel,
	  cdreseau,
      dateprel,
	FROM   
      edc_prelevements
),

resultats AS (
    SELECT
      referenceprel,
      cdparametresiseeaux,
      valtraduite,
      limitequal,
      CAST(regexp_extract(REPLACE(limitequal, ',', '.'), '-?\d+(\.\d+)?') AS FLOAT) AS limitequal_float,
      regexp_extract(limitequal, '[a-zA-Zµg]+/?[a-zA-Z/L]+$') AS unite,
    FROM  
        edc_resultats 
    ),

mapping_categories_simple AS (
    SELECT
      cdparametresiseeaux  ,
      STRING_AGG(DISTINCT categorie) AS categorie,
    FROM 
      mapping_categories
    GROUP BY 
        cdparametresiseeaux
     HAVING 
        COUNT(DISTINCT categorie) = 1
),

mesures_cat AS (
    SELECT
        resultats.*,
        prelevement.dateprel,
        mapping_categories_simple.categorie
    FROM 
        resultats
    LEFT JOIN
    	prelevement
    ON
       resultats.referenceprel = prelevement.referenceprel
    LEFT JOIN
        udi_cog
    ON
    	udi_cog.cdreseau = prelevement.cdreseau
    LEFT JOIN 
        mapping_categories_simple
    ON
        resultats.cdparametresiseeaux = mapping_categories_simple.cdparametresiseeaux
    )
    
SELECT 
 COUNT(*)
FROM
 mesures_cat
"""

nb_result_2 = con.sql(query2).df()

if nb_result_2.iloc[0].values == nb_result_ref.iloc[0].values:
    print("ok :)")
else:
    print("KO")
    print("diff: ", nb_result_2.iloc[0].values - nb_result_ref.iloc[0].values)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Vinca\AppData\Local\Temp\ipykernel_14360\481890040.py:1: SyntaxWarning: invalid escape sequence '\d'
  query2 = """


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

KO
diff:  [1784283587]


# ZOOM uid/cog

In [9]:
query_udi_cog = """
WITH
udi AS (
    SELECT
      inseecommune AS commune_code_insee,
      cdreseau,
    FROM
      edc_communes
),

cog AS (
    SELECT
      DEP AS code_departement,
      REG AS code_region,
      COM AS commune_code_insee,
    FROM 
      cog_communes
),

udi_cog AS (
    SELECT
      udi.commune_code_insee ,
      udi.cdreseau,
      cog.code_departement,
      cog.code_region
    FROM
      udi
    LEFT JOIN 
      cog
    ON 
      udi.commune_code_insee = cog.commune_code_insee
)
     
SELECT 
 cdreseau,
 COUNT(*)
FROM
 udi_cog
GROUP BY 1
ORDER BY 2 DESC
"""

udi_cog_df = con.sql(query_udi_cog).df()
udi_cog_df

,cdreseau,count_star()
0,048000295,490
1,048000516,440
2,048000698,430
3,032000434,399
4,048005357,370
...,...,...
23762,063008169,1
23763,063008223,1
23764,068006567,1
23765,071004037,1


In [10]:
query_udi_cog = """
WITH
udi AS (
    SELECT
      inseecommune AS commune_code_insee,
      cdreseau,
    FROM
      edc_communes
),

cog AS (
    SELECT
      DEP AS code_departement,
      REG AS code_region,
      COM AS commune_code_insee,
    FROM 
      cog_communes
),

udi_cog AS (
    SELECT
      udi.commune_code_insee ,
      udi.cdreseau,
      cog.code_departement,
      cog.code_region
    FROM
      udi
    LEFT JOIN 
      cog
    ON 
      udi.commune_code_insee = cog.commune_code_insee
)
     
SELECT 
 cdreseau,
 COUNT(*)
FROM
 udi_cog
GROUP BY 1
ORDER BY 2 DESC
"""

udi_cog_df = con.sql(query_udi_cog).df()
udi_cog_df

,cdreseau,count_star()
0,048000295,490
1,048000516,440
2,048000698,430
3,032000434,399
4,048005357,370
...,...,...
23762,071004035,1
23763,073008248,1
23764,077004841,1
23765,081004251,1


In [11]:
query_test = """
SELECT
      inseecommune AS commune_code_insee,
      cdreseau,
    FROM
      edc_communes
   WHERE 
      cdreseau = '048000295'
"""
con.sql(query_test).df()

,commune_code_insee,cdreseau
0,48016,048000295
1,48016,048000295
2,48016,048000295
3,48016,048000295
4,48034,048000295
...,...,...
400,48147,048000295
401,48147,048000295
402,48147,048000295
403,48147,048000295


In [12]:
query_udi_cog = """
WITH
udi AS (
    SELECT
      inseecommune AS commune_code_insee,
      cdreseau,
    FROM
      edc_communes
),

cog AS (
    SELECT
      DEP AS code_departement,
      REG AS code_region,
      COM AS commune_code_insee,
    FROM 
      cog_communes
),

udi_cog AS (
    SELECT
      udi.commune_code_insee ,
      udi.cdreseau,
      cog.code_departement,
      cog.code_region
    FROM
      udi
    LEFT JOIN 
      cog
    ON 
      udi.commune_code_insee = cog.commune_code_insee
)
     
SELECT 
 commune_code_insee,
 COUNT(cdreseau)
FROM
 udi_cog
GROUP BY 1
ORDER BY 2 DESC
"""

udi_cog_df = con.sql(query_udi_cog).df()
udi_cog_df

,commune_code_insee,count(cdreseau)
0,48009,1140
1,48127,588
2,48146,490
3,48094,480
4,48027,448
...,...,...
34909,15047,1
34910,87155,1
34911,30317,1
34912,15031,1


In [13]:
query_test = """
SELECT
      inseecommune AS commune_code_insee,
      cdreseau,
    FROM
      edc_communes
   WHERE 
      commune_code_insee = '48009'
"""
con.sql(query_test).df()

,commune_code_insee,cdreseau
0,48009,048000698
1,48009,048000698
2,48009,048000698
3,48009,048000698
4,48009,048000698
...,...,...
565,48009,048002314
566,48009,048002314
567,48009,048002314
568,48009,048003668
